API CLIMA BRASIL

In [ ]:
import pandas as pd
import requests
from datetime import datetime
import time

def format_datetime(data, hora):
    try:
        dt = datetime.strptime(f"{data} {hora}", "%d/%m/%Y %H:%M")
        return dt.strftime("%Y-%m-%dT%H:00")
    except Exception as e:
        print(f"Erro formatando data/hora: {e}")
        return None

def get_lat_lon(cidade):
    url = f"https://geocoding-api.open-meteo.com/v1/search?name={cidade}&count=1&language=pt&format=json"
    try:
        r = requests.get(url)
        dados = r.json()
        if 'results' in dados and len(dados['results']) > 0:
            return dados['results'][0]['latitude'], dados['results'][0]['longitude']
        else:
            print(f"Cidade não encontrada: {cidade}")
            return None, None
    except Exception as e:
        print(f"Erro geocodificando {cidade}: {e}")
        return None, None

def get_weather_open_meteo(lat, lon, data_iso):
    url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={lat}&longitude={lon}"
        f"&hourly=temperature_2m,relative_humidity_2m,precipitation,windspeed_10m"
        f"&start={data_iso}&end={data_iso}&timezone=America/Sao_Paulo"
    )

    try:
        r = requests.get(url)
        dados = r.json()

        if "hourly" in dados:
            return {
                "Temperatura (°C)": dados["hourly"]["temperature_2m"][0],
                "Umidade (%)": dados["hourly"]["relative_humidity_2m"][0],
                "Precipitação (mm)": dados["hourly"]["precipitation"][0],
                "Vento (km/h)": dados["hourly"]["windspeed_10m"][0],
            }
        else:
            return None
    except Exception as e:
        print(f"Erro buscando clima em {lat},{lon} {data_iso}: {e}")
        return None

def main():
    df = pd.read_csv("cidades_partidas_corrigido.csv")
    clima_list = []

    for i, row in df.iterrows():
        cidade = row["cidade"]
        data = row["data"]
        hora = row["hora"]

        data_iso = format_datetime(data, hora)
        if not data_iso:
            clima_list.append({})
            continue

        lat, lon = get_lat_lon(cidade)
        if lat is None or lon is None:
            clima_list.append({})
            continue

        print(f"🌐 Buscando clima de {cidade} em {data_iso}")
        clima = get_weather_open_meteo(lat, lon, data_iso)
        clima_list.append(clima if clima else {})

        time.sleep(0.6)  # polidez com a API

    clima_df = pd.DataFrame(clima_list)
    df_final = pd.concat([df, clima_df], axis=1)
    df_final.to_csv("jogos_com_clima_open_meteo.csv", index=False, encoding='utf-8-sig')
    print("Arquivo salvo como jogos_com_clima_open_meteo.csv")

if __name__ == "__main__":
    main()


🌐 Buscando clima de Salvador em 2024-04-14T18:00
🌐 Buscando clima de Criciuma em 2024-04-13T18:00
🌐 Buscando clima de Porto Alegre em 2024-04-13T18:00
🌐 Buscando clima de Rio de Janeiro em 2024-04-13T21:00
🌐 Buscando clima de Sao Paulo em 2024-04-13T21:00
🌐 Buscando clima de Curitiba em 2024-04-14T16:00
🌐 Buscando clima de Goiania em 2024-04-14T16:00
🌐 Buscando clima de Sao Paulo em 2024-04-14T16:00
🌐 Buscando clima de Rio de Janeiro em 2024-04-14T16:00
🌐 Buscando clima de Belo Horizonte em 2024-04-14T17:00
🌐 Buscando clima de Rio de Janeiro em 2024-04-18T21:00
🌐 Buscando clima de Salvador em 2024-04-16T21:00
🌐 Buscando clima de Cuiaba em 2024-04-17T16:00
🌐 Buscando clima de Porto Alegre em 2024-04-17T19:00
🌐 Buscando clima de Braganca Paulista em 2024-04-17T19:00
🌐 Buscando clima de Belo Horizonte em 2024-04-17T20:00
🌐 Buscando clima de Fortaleza em 2024-04-17T20:00
🌐 Buscando clima de Caxias do Sul em 2024-04-17T20:00
🌐 Buscando clima de Sao Paulo em 2024-04-17T20:00
🌐 Buscando clima